# importance sampling using mixture proposal distributions for two independent uniform variables

method 1

In [2]:
import numpy as np

# Parameters
n_samples = 100
alpha = 0.8
beta = 0.8

# Target distributions
target_SH_azi_low, target_SH_azi_high = 300, 320
target_SH_low, target_SH_high = 16.2, 19.8

# Proposal mixture components
prop1_SH_azi = (310, 320)
prop2_SH_azi = (300, 310)
prop1_SH = (17, 19.8)
prop2_SH = (16.2, 17)

# ----- Sample from proposals -----
n1_azi = int(alpha * n_samples)
n2_azi = n_samples - n1_azi
n1_SH = int(beta * n_samples)
n2_SH = n_samples - n1_SH

samples_SH_azi = np.concatenate([
    np.random.uniform(*prop1_SH_azi, size=n1_azi),
    np.random.uniform(*prop2_SH_azi, size=n2_azi)
])

samples_SH = np.concatenate([
    np.random.uniform(*prop1_SH, size=n1_SH),
    np.random.uniform(*prop2_SH, size=n2_SH)
])

# Shuffle to mix samples
np.random.shuffle(samples_SH_azi)
np.random.shuffle(samples_SH)

# ----- Compute densities -----
def uniform_pdf(x, low, high):
    return np.where((x >= low) & (x <= high), 1.0 / (high - low), 0.0)

def mixture_uniform_pdf(x, low1, high1, low2, high2, w):
    pdf1 = np.where((x >= low1) & (x <= high1), 1.0 / (high1 - low1), 0.0)
    pdf2 = np.where((x >= low2) & (x <= high2), 1.0 / (high2 - low2), 0.0)
    return w * pdf1 + (1 - w) * pdf2

target_pdf_SH_azi = uniform_pdf(samples_SH_azi, target_SH_azi_low, target_SH_azi_high)
target_pdf_SH = uniform_pdf(samples_SH, target_SH_low, target_SH_high)

proposal_pdf_SH_azi = mixture_uniform_pdf(samples_SH_azi, *prop1_SH_azi, *prop2_SH_azi, alpha)
proposal_pdf_SH = mixture_uniform_pdf(samples_SH, *prop1_SH, *prop2_SH, beta)

# ----- Importance weights -----
weights = (target_pdf_SH_azi * target_pdf_SH) / (proposal_pdf_SH_azi * proposal_pdf_SH)
weights /= np.sum(weights)

# ----- Example output -----
for i in range(10):  # show first 10 samples
    print(f"Sample {i+1:2d}: SH_azi={samples_SH_azi[i]:.2f}, SH={samples_SH[i]:.2f}, weight={weights[i]:.4f}")

print("\nTotal samples:", len(samples_SH))
print("Sum of weights:", np.sum(weights))


Sample  1: SH_azi=309.83, SH=17.69, weight=0.0243
Sample  2: SH_azi=311.50, SH=17.10, weight=0.0061
Sample  3: SH_azi=311.69, SH=19.38, weight=0.0061
Sample  4: SH_azi=313.92, SH=16.34, weight=0.0069
Sample  5: SH_azi=311.24, SH=16.34, weight=0.0069
Sample  6: SH_azi=313.79, SH=17.56, weight=0.0061
Sample  7: SH_azi=316.03, SH=17.39, weight=0.0061
Sample  8: SH_azi=310.82, SH=18.72, weight=0.0061
Sample  9: SH_azi=304.29, SH=18.34, weight=0.0243
Sample 10: SH_azi=303.70, SH=17.14, weight=0.0243

Total samples: 100
Sum of weights: 0.9999999999999999


method 2

In [72]:
import numpy as np

np.random.seed(11)
name_prefix = '250922'
# Parameters
n_samples = 90
alpha = 0.8
beta = 0.8

# --- Target distributions ---
# SH_azi ~ U(300, 320)
target_SH_azi_low, target_SH_azi_high = 300, 320
p_SH_azi = 1/(target_SH_azi_high - target_SH_azi_low)

# SH ~ U(16.2, 19.8)
target_SH_low, target_SH_high = 16.2, 19.8
p_SH = 1/(target_SH_high - target_SH_low)

# --- Proposal mixture components ---
# SH_azi: 0.2 U(300,310) + 0.8 U(310,320)
u1 = np.random.rand(n_samples)
u1_quantile = np.quantile(u1,alpha)
samples_SH_azi = np.empty(n_samples)
samples_SH_azi[u1 > u1_quantile] = np.random.uniform(300, 310, size=(u1 > u1_quantile).sum())
samples_SH_azi[u1 <= u1_quantile] = np.random.uniform(310, 320, size=(u1 <= u1_quantile).sum())
q_SH_azi = np.where((samples_SH_azi >= 310) & (samples_SH_azi < 320), alpha / 10.0, (1 - alpha) / 10.0)
print(f'SH_azi samples min: {np.min(samples_SH_azi):.2f}')
print(f'SH_azi samples max: {np.max(samples_SH_azi):.2f}')

# SH: 0.2 U(16.2,17) + 0.8 U(17,19.8)
u2 = np.random.rand(n_samples)
u2_quantile = np.quantile(u2,beta)
samples_SH = np.empty(n_samples)
samples_SH[u2 > u2_quantile] = np.random.uniform(16.2, 17.0, size=(u2 > u2_quantile).sum())
samples_SH[u2 <= u2_quantile] = np.random.uniform(17.0, 19.8, size=(u2 <= u2_quantile).sum())
q_SH = np.where((samples_SH >= 17) & (samples_SH < 19.8), beta / (19.8 - 17.0), (1 - beta) / (17.0 - 16.2))
print(f'SH samples min: {np.min(samples_SH):.2f}')
print(f'SH samples max: {np.max(samples_SH):.2f}')

# --- Importance weights ---
# Since the two variables are independent, total weight = (p_SH_azi * p_SH) / (q_SH_azi*q_SH)
weights = (p_SH_azi * p_SH) / (q_SH_azi * q_SH)
weights /= np.sum(weights)

importance_sampling = np.column_stack((samples_SH_azi,q_SH_azi,samples_SH,q_SH,weights))

header_string = 'SH_azi,q_SH_azi,SH,q_SH,weights'
np.savetxt(f'data/{name_prefix}_importance_sampling.csv',importance_sampling,delimiter=',',fmt='%.4f',header=header_string,comments='')
print(np.sum(importance_sampling[:,0]>310))
print(np.sum(importance_sampling[:,2]>17))

SH_azi samples min: 300.86
SH_azi samples max: 319.89
SH samples min: 16.26
SH samples max: 19.73
72
72


In [33]:
print(np.sum(importance_sampling[:,0]>310))
print(np.sum(importance_sampling[:,2]>17))

72
75
